<a href="https://colab.research.google.com/github/dtht2d/bispectrum_component/blob/main/codes/optimization/Wigner_D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Code optimization:**  Compile Wigner D with Numba

**Numba**
- An open source JIT compiler that translates a subset of Python and NumPy code into fast machine code.
- Designed to be used with NumPy arrays and functions. Numba generates specialized code for different array data types and layouts to optimize performance.


In [1]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install numba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
time: 530 µs (started: 2023-02-23 05:18:24 +00:00)


In [4]:
import numba as nb
from numba import jit, complex128, float64
import numpy as np
import cmath


@jit(complex128(float64, float64, float64, float64, float64, float64), fastmath=True, cache=True, nogil=True)
def wigner_D(j, m, mp, theta_0, theta, phi):
    """
    This function calculates the Wigner D function for a given set of input parameters.

    Args:
        j (scalar): angular momentum
        m (scalar): eigenvalue of angular momentum
        mp (scalar): eigenvalue of j along rotated axis
        theta_0 (scalar): fist angle of rotation [0,pi]
        theta (scalar): second angle of rotation [0,pi]
        phi (scalar): third angle of rotation [0,2pi]

    Returns:
        wigner D function - complex number

    ==========================Reference==================================
    [5] Chapter 4.3-(p.76,eq.1)  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
    """
  
    def fact(n):
        """
        This function is used to calculate factorial of a number by using an 
        interative approach instead of recursive approach
        """
        result = 1
        for i in range(1, n + 1):
          result *= i
        return result
    def compute_dsmall():
        """
        Calculate the Wigner d small- real function involving trigonometric functions.

        Returns:
            wigner d - real function.

        References:
        [5] Chapter 4.3.1-(p.76,eq.4)  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        """
        kmax = max(0, m - mp)
        kmin = min(j + m, j - mp)
        term1 = np.sqrt(fact(j + m) * fact(j - m) * fact(j + mp) * fact(j - mp))
        total_sum = 0
        for k in range(kmin, kmax+1):
            numerator = (-1) ** k * (cmath.cos(theta / 2)) ** (2 * j - 2 * k + m - mp) *\
                        (cmath.sin(theta / 2)) ** (2 * k - m + mp)
            denominator = fact(k) * fact(j + m - k) * fact(j - mp - k) * fact(mp - m + k)
            total_sum += numerator / denominator
        return term1 * total_sum
    term1 = np.exp(-1j * m * theta_0)
    term2 = compute_dsmall()
    term3 = np.exp(-1j * mp * phi)
    result = term1 * term2 * term3
    return result



time: 4.3 s (started: 2023-02-23 05:18:25 +00:00)


**Example**

In [ ]:
import numpy as np
j, m, mp, theta_0, theta,phi= 1, 1, 0, np.pi, np.pi/2, 0
# Calculate the Wigner D function using Numba
result = wigner_D(j, m, mp, theta_0, theta, phi)
print(result)